In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pickle
import ipywidgets as widgets
from scipy.stats import norm
import time

import sys
import os
sys.path.insert(0, os.path.abspath('Practical03_Support'))

from ece4078.localization.dynamic_system_import import import_based_on_version
import_based_on_version(globals(), locals())
from ece4078.localization.Renderer import *

from Measurements import *
from Distribution import *
from Helper import *
from SLAM_plotting import *

# Model Uncertainty: Why?

Recall our PenguinPi model kinematic model from lecture 2

<img src="https://i.postimg.cc/g0L3DYV4/Penguin-Pi-Model.png" width="300" height="300" align="center">

Let's see how well this model predicts the real robot motion. To do so, we will apply control inputs to the real robot and measure the resulting state. We will then apply the same control inputs into our motion model and observe whether there is a difference between the observed (red curve) and predicted robot states (blue curve).

**Note**: The real data was generated by applying varying left and right wheel inputs to the PenguinPi and measuring where the robot ended up with an overhead camera.

In [ ]:
# Create a robot
bot = PenguinPi(wheels_width=0.14, wheels_radius=0.01)

# Here we load our data including the control inputs and the true state 
with open('Practical03_Support/data/data.dat', "rb") as f:
    data = pickle.load(f)
    
true_state = data['state'][48:,:]
control = data['control'][48:,:]

# Set the initial state of the robot from the data
init_x, init_y, init_theta = true_state[0,:]
bot.set_state(x=init_x, y=init_y, theta=init_theta)

# Set the time step of the similation
dt = 0.2
# Here we will predict the robot's state using the model 
state = np.zeros((control.shape[0],3))

# At each timestep we apply the same control that was applied to the
# real robot to our model
for c in range(control.shape[0]):
    state[c,:] = np.array(bot.get_state()) 
    drive_signal = DriveMeasurement(control[c,0],control[c,1],dt)
    bot.drive(drive_signal)

#Create a visualizer and visualize the predicted and the true state
rend = Renderer.Instance()
rend.initialize(state, true_state=true_state)


# Model Uncertainty: How?

## 1. Propagation of Uncertainty

Given the initial known position of our 1D robot ($x_t$), after each step, our uncertainty about the robot's new state is defined as follows:

<center>
$\begin{equation}
\begin{split}
& P(x_{t+1}=x_{t}+1) = 0.5, \\
& P(x_{t+1}=x_{t}+1.1) = 0.25, \\
& P(x_{t+1}=x_{t}+0.9) = 0.25. 
\end{split}
\end{equation}$
</center>

Below we show how our uncertainty about the robot's true position changes with every new step.

**Interaction**
- Observe how our uncertainty about the robot's position changes when the robot moves forward 5 steps

In [ ]:
# We start with a known position: probability 1.0 at position 10
plt.ion()
belief_position = Distribution.unit_pulse(10)
x, y = belief_position.plotlists(0, 500)
old_line,line = createInitialPlot(x, y)

def convolve_distribution(b=None):
    global belief_position
    
    # For each new control:
    # - there is 50% chance the robot moves 1m
    # - there is 25% chance the robot moves 1.1m
    # - there is 25% change the robot moves 0.9m
    
    # Plot prior belief
    old_x, old_y = belief_position.plotlists(0, 500)
    old_line.set_data(old_x, old_y)
    
    # We enconde this belief into a new distribution
    for i in range(5):
        move_distribution = Distribution.triangle(1,2)
        # We modify our belief about the robots position with a convolution
        belief_position = belief_position.convolve(move_distribution)
        new_x, new_y = belief_position.plotlists(0, 500)
        line.set_data(new_x, new_y)
        
def reset(b=None):
    global belief_position
    belief_position = Distribution.unit_pulse(10)
    x, y = belief_position.plotlists(0, 500)
    old_line.set_data(x,y)
    line.set_data(x, y)
    
btn_move, btn_reset = createbtns()
btn_move.on_click(convolve_distribution)
btn_reset.on_click(reset)

## 2. Reducing Uncertainty with Measurements - 1D Kalman Filter

Lets see how we can combine our model predictions and the noisy measurements from the robot's sensor in order to reduce our uncertainty about the robot's position. 

The robot model is defined as $x_{k+1} = ax_k + bu_k + \epsilon_Q, \epsilon_Q \sim \mathcal{N}(0, \sigma^2_Q)$. Similarly, we define the measurement model as $z_k = cx_k + \epsilon_R, \epsilon_R \sim \mathcal{N}(0, \sigma^2_R)$. 

### Step 1: 
We generate the true states and the noisy measurements associated to those states.

In [ ]:
# 1D model parameters
u = 100 # We will drive our robot with a constant control input
a = 1
b = 1

# Number of measurements we want to generate
steps = 10

# This is our measurement constant
C = 1

#This is our true noise
true_move_noise = 2
true_mes_noise = 10

true_state, measurements = generate_measurements(steps=steps, a=a, b=b, u=u, c=C, true_move_noise=true_move_noise,
                                                true_mes_noise=true_mes_noise)

### Step 2: 
Setup and run our Kalman Filter

**Interaction**:
- Define different ``sigma_Q`` and ``sigma_R`` values and observe their effect on the Kalman filter's estimates

**TODO**:
- Complete the update state step

In [ ]:
#----------------------------------PARAMETERS--------------------------------------
# This is our mean at t, we initialize at zero
mu_k = 0
# This is our covariance at t, we initialize very small implying we know x0 very well
sigma_k = 0.4
# This is the uncertainty in our move function 
sigma_Q = 2
# This is what we think our measurement noise is
sigma_R = 10
#-----------------------------------------------------------------------------------

# We will save our predicted state here
pred_state = np.zeros((steps,1)) 
# We will save our estimated state here
est_state = np.zeros((steps,1))

# Plotting code
x = np.linspace(0,1000,2000)
fig, ax, hfig = create_slam_plot(x, mu_k, sigma_k, true_state, sigma_R)

for i in range(steps-1):
    
    # Prediction step
    mu_bar = a*mu_k + b*u
    sigma_bar = np.sqrt(a*a*sigma_k*sigma_k + sigma_Q*sigma_Q)  

    # Update step
    z = measurements[i+1]
    #TODO: Complete this step -----------------------------------------------------
    K = 0
    mu_k = 0
    sigma_k = 0
    #ENDTODO ----------------------------------------------------------------------

    # Plot state distribution based on model prediction (this is our prior)
    y_pred = norm.pdf(np.array(x), loc=mu_bar, scale=sigma_bar)

    # Here we compute the likelihood of the robot's position given a new measurement 
    # Note we plot the measurement centered around the true measured value
    y_mes = norm.pdf(np.array(x), loc=measurements[i+1], scale=sigma_R)

    # We combine the knowledge from our model and the likelihood given our noisy measurement
    y_est = norm.pdf(np.array(x), loc=mu_k, scale=sigma_k)

    # Save our estimates
    pred_state[i+1] = mu_bar
    est_state[i+1] = mu_k
    
    update_slam_plot(fig, ax, hfig, x, true_state[i+1], y_pred, y_mes, y_est)
    
plt.close(fig)
lb1 = widgets.Label(value='Average abs. error with measurements' + 
                    " only is {}".format(np.mean(np.abs(true_state[:-1]-measurements[:-1]))))
    
lb2 = widgets.Label(value=r'Average abs. error with Kalman Filter' + 
                    " is {}".format(np.mean(np.abs(true_state[:-1]-est_state[:-1]))))

info = widgets.VBox([lb1, lb2])
display(info)

## 3. Extended Kalman Filter

Lets now go back to our PenguinPi robot. We wish to have a better estimate of the PenguinPi robot's state. To do so, we will use an extended Kalman Filter (EKF). 

Using EKF, we will combine the model predictions and the measurements that our robot obtains with its camera in order to better estimate the robot's state.

For our implementation, we define the robot model as:

$\boldsymbol{x}_{k+1} = A\boldsymbol{x}_k + B\boldsymbol{u}_k + \boldsymbol{w}$, with $\boldsymbol{w} \sim \mathcal{N}(\boldsymbol{0}, \Sigma_{Q})$.

Similarly, we define the measurement model as 

$\boldsymbol{y}_{k} = C\boldsymbol{x}_k + \boldsymbol{v}$, with $\boldsymbol{v} \sim \mathcal{N}(\boldsymbol{0}, \Sigma_{R})$.

### Step 1:

Below we show the PenguinPi state when only the motion model is used to estimate the robot's state. Since we are only considering the model's prediction, we see that the robot's trajectory significantly differs from the desired trajectory.

In [ ]:
#Here we load our data including the true state 
with open('Practical03_Support/data/data_ekf.dat', "rb") as f:
    data = pickle.load(f)
    
gt_state = data['state']
control = data['control']
measurements = data['measurement']

#Create a robot and place it at the true starting position and orientation 
bot = PenguinPi(0.15, 0.01)
true_state = data['state']
control = data['control']

# Set the initial state of the robot from the data
init_x, init_y, init_theta = true_state[0,:]
bot.set_state(x=init_x, y=init_y, theta=init_theta)

#Apply our control to the robot and visualize
#Our timestamp is always 0.2
dt = 0.2
#Here we will save our state variable 
state = np.zeros((control.shape[0],3))

for c in range(control.shape[0]):
    state[c,:] = np.array(bot.get_state())
    drive_signal = DriveMeasurement(control[c,0],control[c,1],dt)
    bot.drive(drive_signal)

#Create a visualizer and visualize our atate
rend = Renderer.Instance()
rend.initialize(state, true_state = gt_state)

### Step 2: 

Setup and run our Extended Kalman Filter. In the plot, the yellow arrows indicate the robot's sensors.

**Interaction**: 

Increase and decrease the uncertainty on the measurements' model ``sigma_measure``. Observe how the uncertainty about the robot's state changes.

In [ ]:
# Setup our robot
bot = PenguinPi(0.14, 0.01)
true_state = data['state']
control = data['control']

# Set the initial state of the robot from the data
init_x, init_y, init_theta = true_state[0,:]
bot.set_state(x=init_x, y=init_y, theta=init_theta)

# Here are our true marker positions
markers = data['markers']

#----------------------------------PARAMETERS--------------------------------------
# Initial uncertainty of our robot
P = np.eye(3)*100
# Uncertainty in our left and right wheel motors
sigma_drive = np.array([10,10])
# Uncertainty in measurement
sigma_measure = np.eye(2)*100
#-----------------------------------------------------------------------------------

# Apply our control to the robot and visualize
# Our timestamp is always 0.2
dt = 0.2

create_video(bot, data, P, sigma_drive, sigma_measure, dt, skip = 1)